In [1]:
from functools import reduce
import BLAST_prob
from BLAST_prob import seq_query, Trie, Hit

# Read sequence and generate query

In [2]:
SEQUENCE,SEQ_PROB, NEUC_PROB, Q_PROB = seq_query.read_sequence(demo_seq_len = 100)
print("first position in the sequence: ", NEUC_PROB[0])
print("overall prob for each nucleotide: ", Q_PROB)


first position in the sequence:  {'A': 0.003333333333333336, 'C': 0.99, 'G': 0.003333333333333336, 'T': 0.003333333333333336}
overall prob for each nucleotide:  {'A': 0.2542000000000002, 'C': 0.36926666666666685, 'G': 0.1794, 'T': 0.19713333333333338}


In [3]:
start_pos, query = seq_query.generate_query(SEQUENCE,SEQ_PROB,query_len=10)
print("The query is taken from sequence position " + str(start_pos) + " to " + str(start_pos+10-1))
print("The generated query: ", query)
print("The corresponding part in the original seq: ", "".join(SEQUENCE[start_pos:(start_pos+10)]))


The query is taken from sequence position 16 to 25
The generated query:  CTGTTTCCAC
The corresponding part in the original seq:  CTGTCTCCAC


# Possible_Matching and the search trie
(0, 4) => the word at this leaf starts at position 0 and scores 4
In this demo case, all eligible words are the exact matches (score = 4), given the scoring scheme, threshold, and the word length.
Change word_len and/or threshold to see the effect

In [4]:
TRIE = Trie.possible_matching(query=query,word_len=4,threshold=2)
print("Result - search trie: ",TRIE.trie)

Finding candidates done
Query words done
Qualified
Begin to build the search trie
There are  7 possible matchings
Branches created
Record scores done
Result - search trie:  {'T': {'C': {'C': {'A': [(5, 4)]}}, 'G': {'T': {'T': [(1, 4)]}}, 'T': {'C': {'C': [(4, 4)]}, 'T': {'C': [(3, 4)]}}}, 'C': {'C': {'A': {'C': [(6, 4)]}}, 'T': {'G': {'T': [(0, 4)]}}}, 'G': {'T': {'T': {'T': [(2, 4)]}}}}


# Initial hit (One-hit)

In [5]:
ONEHIT = Hit.find_one_hit(search_trie=TRIE, sequence_prob=NEUC_PROB, word_len=4,query=query,score_threshold=2)
for h in ONEHIT:
    print(h)

The hit matches 7 and 10 in the database sequence with the positions from 6 to 9 in query sequence with score 3.9699999999999998
The hit matches 10 and 13 in the database sequence with the positions from 6 to 9 in query sequence with score 3.9699999999999998
The hit matches 13 and 16 in the database sequence with the positions from 6 to 9 in query sequence with score 2.9733333333333336
The hit matches 14 and 17 in the database sequence with the positions from 6 to 9 in query sequence with score 2.033333333333333
The hit matches 14 and 17 in the database sequence with the positions from 5 to 8 in query sequence with score 2.033333333333333
The hit matches 15 and 18 in the database sequence with the positions from 6 to 9 in query sequence with score 2.0366666666666666
The hit matches 16 and 19 in the database sequence with the positions from 0 to 3 in query sequence with score 3.8
The hit matches 17 and 20 in the database sequence with the positions from 1 to 4 in query sequence with sco

In [6]:
# Two-hit
TWOHIT = Hit.find_two_hit(sequence_prob=NEUC_PROB, query=query, hits=ONEHIT, window=15, score_threshold=2)
print("two-hits:")
for h in TWOHIT:
    print(h)

two-hits:
The hit matches 16 and 23 in the database sequence with the positions from 0 to 7 in query sequence with score 6.763333333333333
The hit matches 16 and 24 in the database sequence with the positions from 0 to 8 in query sequence with score 5.77
The hit matches 16 and 25 in the database sequence with the positions from 0 to 9 in query sequence with score 6.72
The hit matches 17 and 24 in the database sequence with the positions from 1 to 8 in query sequence with score 6.773333333333333
The hit matches 17 and 25 in the database sequence with the positions from 1 to 9 in query sequence with score 7.723333333333333
The hit matches 18 and 25 in the database sequence with the positions from 2 to 9 in query sequence with score 6.913333333333333


# Ungapped extension

In [7]:
UNGAP_HIT = Hit.ungapped_extension(TWOHIT, NEUC_PROB,query)
for hit in UNGAP_HIT:
    print(hit)

The hit matches 16 and 25 in the database sequence with the positions from 0 to 9 in query sequence with score 8.733333333333333
The hit matches 16 and 25 in the database sequence with the positions from 0 to 9 in query sequence with score 6.739999999999999
The hit matches 16 and 25 in the database sequence with the positions from 0 to 9 in query sequence with score 6.72
The hit matches 16 and 25 in the database sequence with the positions from 0 to 9 in query sequence with score 8.713333333333333
The hit matches 16 and 25 in the database sequence with the positions from 0 to 9 in query sequence with score 8.693333333333333
The hit matches 16 and 25 in the database sequence with the positions from 0 to 9 in query sequence with score 8.433333333333334


# Gapped extension

In [8]:
GAPPED_HIT = Hit.gapped_extension(UNGAP_HIT, NEUC_PROB,query)
for hit in GAPPED_HIT:
    print(hit)

The hit matches 16 and 25 in the database sequence with the positions from 0 to 9 in query sequence with score 8.733333333333333
The hit matches 16 and 25 in the database sequence with the positions from 0 to 9 in query sequence with score 6.739999999999999
The hit matches 16 and 25 in the database sequence with the positions from 0 to 9 in query sequence with score 6.72
The hit matches 16 and 25 in the database sequence with the positions from 0 to 9 in query sequence with score 8.713333333333333
The hit matches 16 and 25 in the database sequence with the positions from 0 to 9 in query sequence with score 8.693333333333333
The hit matches 16 and 25 in the database sequence with the positions from 0 to 9 in query sequence with score 8.433333333333334


# Get the hit with the highest score as the final result

In [9]:
MAX_HIT = Hit.find_max_hit(GAPPED_HIT)
print(MAX_HIT)

The hit matches 16 and 25 in the database sequence with the positions from 0 to 9 in query sequence with score 8.733333333333333


Matches the position where the query is generated from :)